In [99]:
import xml.dom.minidom
import uuid
import subprocess
import os
from tqdm import tqdm
import random 
import shutil 

In [101]:
# consts
SCENE_FILE_MASTER = 'master.xml'
DATA_SET_ROOT = '/home/ax/master/2019_02/masterThesis/DeepExplain/experiments/data/generated'
master_doc = xml.dom.minidom.parse(SCENE_FILE_MASTER)
SAMPLES = 10
X_MIN = -100
X_MAX = 100
Z_MIN = -1000
Z_MAX = 0
ANGLE_MIN = -60
ANGLE_MAX = 60

master_requirements = ['RUST_3d_Low1.serialized', 'RUST_3d_Low1_Difuse.jpg', 'RUST_3d_Low1_Bump.jpg']

for file in master_requirements:
    shutil.copy(file, f'{DATA_SET_ROOT}/{file}')
# requirements for mitsuba
my_env = os.environ.copy()
my_env["LD_LIBRARY_PATH"] = "/home/ax/Programs/mitsuba/dist:" + my_env["LD_LIBRARY_PATH"]
my_env["PATH"] = "/home/ax/Programs/mitsuba/dist:" + my_env["PATH"]

In [102]:


# compute a scene string
def get_scene_xml_string(doc, x, z, angle):
    
    shape_node_human = doc.getElementsByTagName('shape')[0]
    transform_node = shape_node_human.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[0]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    
    shape_node_rect = doc.getElementsByTagName('shape')[1]
    transform_node = shape_node_rect.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[1]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    
    return doc.toxml()

# save the scene to file
def save_scene(scene_str, path=DATA_SET_ROOT):
    file_name = f'{path}/a_{uuid.uuid4()}.xml'
    with open(file_name, "w") as f:
        f.write(scene_str)
    return file_name

# 
def render_scene(filename):
    cmd = ["/home/ax/Programs/mitsuba/build/release/mitsuba/mitsuba", filename]
    cmd_out = subprocess.run(cmd, check=True, env=my_env)
    return filename.replace('xml', 'exr')
    
def convert_exr_to_png(filename):
    cmd = ["convert", filename, '-colorspace', 'RGB', '-colorspace', 'sRGB', filename.replace('exr', 'png')]
    cmd_out = subprocess.run(cmd, check=True, env=my_env)
    
def clean_up(filename):
    os.remove(filename)
    
def generate_sample(master_doc, x, z, angle):
    # generate xml scene string
    scene_str = get_scene_xml_string(master_doc, x, z, angle)
    # save scene as file
    scene_file_name = save_scene(scene_str)
    # render scene to exr
    result_file_name = render_scene(scene_file_name)
    # convert to png
    convert_exr_to_png(result_file_name)
    # delete exr
    clean_up(result_file_name)

In [103]:
for i in tqdm(range(SAMPLES)):
    x = random.uniform(X_MIN, X_MAX)
    z = random.uniform(Z_MIN, Z_MAX)
    angle = random.uniform(ANGLE_MIN, ANGLE_MAX)
    generate_sample(master_doc, x, z, angle)


100%|██████████| 10/10 [00:57<00:00,  5.75s/it]
